In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Concatenate, Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [3]:
for layer in vgg16.layers:
    layer.trainable = False
for layer in resnet50.layers:
    layer.trainable = False

In [4]:
input_layer = Input(shape=(224, 224, 3))

In [5]:
vgg_output = vgg16(input_layer)
resnet_output = resnet50(input_layer)


In [6]:
vgg_output = Flatten()(vgg_output)
resnet_output = Flatten()(resnet_output)
concatenated = Concatenate()([vgg_output, resnet_output])



In [7]:
train_resnet_datagen = ImageDataGenerator(rescale=1./255)

NameError: name 'num_classes' is not defined

In [14]:
train_resnet_generator = train_resnet_datagen.flow_from_directory(
        'PlantVillage',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 4500 images belonging to 3 classes.


In [15]:
val_resnet_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
val_resnet_generator = val_resnet_datagen.flow_from_directory(
        'PlantVillage',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 4500 images belonging to 3 classes.


In [17]:
train_vgg_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_vgg_generator = train_vgg_datagen.flow_from_directory(
        'PlantVillage',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 4500 images belonging to 3 classes.


In [19]:
val_vgg_datagen = ImageDataGenerator(rescale=1./255)

In [20]:
val_vgg_generator = val_vgg_datagen.flow_from_directory(
        'PlantVillage',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 4500 images belonging to 3 classes.


In [31]:
train_resnet_data = train_resnet_generator.next()
train_vgg_data = train_vgg_generator.next()
train_labels = train_generator.labels


In [32]:
val_resnet_data = val_resnet_generator.next()
val_vgg_data = val_vgg_generator.next()
val_labels = validation_generator.labels


In [21]:
train_data = (train_resnet_generator, train_vgg_generator)
val_data = (val_resnet_generator, val_vgg_generator)


In [39]:
train_resnet_data = np.concatenate([np.array(item)[np.newaxis, :] for item in train_resnet_data], axis=0)
train_vgg_data = np.concatenate([np.array(item)[np.newaxis, :] for item in train_vgg_data], axis=0)



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 5 dimension(s) and the array at index 1 has 3 dimension(s)

In [35]:
print(train_resnet_data.shape)
print(train_vgg_data.shape)
print(train_labels.shape)


AttributeError: 'tuple' object has no attribute 'shape'

In [22]:
val_labels = []
for i in range(len(val_resnet_generator.classes)):
    if val_resnet_generator.classes[i] == val_vgg_generator.classes[i]:
        val_labels.append(val_resnet_generator.classes[i])
    else:
        val_labels.append(-1)


In [23]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_output = resnet_model.output
resnet_output = Flatten()(resnet_output)

In [24]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_output = vgg_model.output
vgg_output = Flatten()(vgg_output)

In [25]:
concatenated_output = concatenate([resnet_output, vgg_output])


In [26]:
dense_layer = Dense(256, activation='relu')(concatenated_output)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(38, activation='softmax')(dropout_layer)


In [27]:
model = Model(inputs=[resnet_model.input, vgg_model.input], outputs=output_layer)


In [28]:
for layer in resnet_model.layers:
    layer.trainable = False

for layer in vgg_model.layers:
    layer.trainable = False

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
history = model.fit(
    x=[train_resnet_data, train_vgg_data],
    y=train_labels,
    epochs=10,
    validation_data=([val_resnet_data, val_vgg_data], val_labels)
)


ValueError: Data cardinality is ambiguous:
  x sizes: 32, 32, 32, 32
  y sizes: 4500
Make sure all arrays contain the same number of samples.

NameError: name 'validation_labels' is not defined